# 0. 농가 정보 가져오기

In [29]:
import requests
import json
farm_code= [81,9,21,25,27,28,29,31,32,39,42,44,45,48,50,51,
            53,54,56,57,59,201,202,204,205,206,207,209,210,
            315,316,318,319,320,324,325,326,327,332,339,344,
            345,349]

url = 'http://apis.data.go.kr/1390000/SmartFarmDATA'

def envdatarqst(farm_id, pgsize = 1):
    serviceKey = '0Aw%2F%2FXJZ308HyMJ7lNciZDyPgD%2FEgpNUnGgZvrlschps7e7hdW2g5NfbU42WXs1QG%2FLZweTcxlvu7trSWQ%2Fc6w%3D%3D'
    uri = url + '/envdatarqst'
    data = '?serviceKey=' + serviceKey + '&searchFrmhsCode=' + str(farm_id) + '&returnType=json' + '&pageSize=' + str(pgsize)
    return requests.get(uri+data).json()

def grwdatarqst(farm_id, pgsize = 1):
    serviceKey = '0Aw%2F%2FXJZ308HyMJ7lNciZDyPgD%2FEgpNUnGgZvrlschps7e7hdW2g5NfbU42WXs1QG%2FLZweTcxlvu7trSWQ%2Fc6w%3D%3D'
    uri = url + '/grwdatarqst'
    data = '?serviceKey=' + serviceKey + '&searchFrmhsCode=' + str(farm_id) + '&returnType=json' + '&pageSize=' + str(pgsize)
    return requests.get(uri+data).json()

def prddatarqst(farm_id, pgsize = 1):
    serviceKey = '0Aw%2F%2FXJZ308HyMJ7lNciZDyPgD%2FEgpNUnGgZvrlschps7e7hdW2g5NfbU42WXs1QG%2FLZweTcxlvu7trSWQ%2Fc6w%3D%3D'
    uri = url + '/prddatarqst'
    data = '?serviceKey=' + serviceKey + '&searchFrmhsCode=' + str(farm_id) + '&returnType=json' + '&pageSize=' + str(pgsize)
    return requests.get(uri+data).json()

In [30]:
import pandas as pd
import numpy as np

def save_csv(fun):
    df_lst = []
    for _ in farm_code:
        pgsize = fun(_)['response']['body']['totalCount']
        tmp = fun(_,pgsize)['response']['body']['items']['item']
        for t in tmp:
            t['id'] = _
        tb = pd.DataFrame.from_dict(tmp)
        df_lst.append(tb)
    df = pd.concat(df_lst)
    tmp_lst = df.columns.tolist()
    col_lst = ['id'] + tmp_lst[:-1]
    df = df[col_lst]
    df.to_csv("{}.csv".format(fun.__name__))
    
save_csv(envdatarqst)
save_csv(grwdatarqst)
save_csv(prddatarqst)

# 1. 데이터 전처리

In [1]:
import pandas as pd

env = pd.read_csv('data/envdatarqst.csv', index_col=0)
grw = pd.read_csv('data/grwdatarqst.csv', index_col=0)
prd = pd.read_csv('data/prddatarqst.csv', index_col=0)

grw_prd = pd.merge(grw, prd, left_on=('id','frmWeek'), right_on=('id','frmWeek'), how='inner')
grw_prd.to_csv("data/{}.csv".format('grw_prd'))

## 1.1 생육정보 합친거 > 배양액

In [2]:
yap = []
for c in list(grw_prd.columns):
    tmp = grw_prd.loc[grw_prd[c] != 0.0]
    if len(tmp) != 0:
        yap.append(c)
grw_prd = grw_prd[yap]
grw_prd # 없는 데이터 지우기

,id,frtstGrupp,flanGrupp,frtstCo,frmhsFclu,hvstGrupp,frmYear_x,frmMonth_x,grwtLt,fcluHg,...,hvstCo,frmWeek,lefCunt,lefBt,stemThck,frmYear_y,frmMonth_y,frmAr,frmhsId_y,outtrn
0,81,0.00,0.0,0.0,0.0,0.0,2017,8,0.0,0.0,...,0.00,31,0.00,0.000,0.0000,2017,8,2700.0,81,0.0
1,81,0.00,0.0,0.0,0.0,0.0,2017,8,0.0,0.0,...,0.00,32,0.00,0.000,0.0000,2017,8,2700.0,81,0.0
2,81,0.00,0.0,0.0,0.0,0.0,2017,8,0.0,0.0,...,0.00,33,0.00,0.000,0.0000,2017,8,2700.0,81,0.0
3,81,0.00,0.0,0.0,0.0,0.0,2017,8,0.0,0.0,...,0.00,34,0.00,0.000,0.0000,2017,8,2700.0,81,0.0
4,81,0.00,0.0,0.0,0.0,0.0,2017,8,0.0,0.0,...,0.00,35,0.00,0.000,0.0000,2017,8,2700.0,81,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1952,349,18.50,0.0,12.0,0.0,15.5,2019,5,0.0,0.0,...,5.25,22,10.75,56.950,8.9150,2019,5,2800.0,349,14579.0
1953,349,18.75,0.0,9.0,0.0,16.5,2019,6,0.0,0.0,...,3.00,23,5.75,59.850,8.3975,2019,6,2800.0,349,5566.0
1954,349,18.50,0.0,4.0,0.0,17.5,2019,6,0.0,0.0,...,5.00,24,5.75,60.875,9.0125,2019,6,2800.0,349,10546.0
1955,349,19.25,0.0,0.0,0.0,18.5,2019,6,0.0,0.0,...,4.00,25,5.75,61.650,8.6550,2019,6,2800.0,349,2507.0


In [5]:
import datetime

qy_week = env.loc[env['otmsuplyqy'] != 0.0] # 1주당 1회 양액 급액량
qy_ph = qy_week.loc[qy_week['ph'] != 0.0] # ph
qy_ec = qy_ph.loc[qy_ph['ec'] != 0.0] # ec
qy_data = qy_ec.loc[qy_ec['cunt'] != 0.0] # 일별 공급 횟수
qy_data = qy_data[['ec', 'ph', 'cunt', 'otmsuplyqy', 'id', 'measDtStr']]
qy_data.to_csv("data/{}.csv".format('qy_data'))
qy_data['measDtStr'] = qy_data['measDtStr'].map(lambda x : list(map(int,[str(x)[:4], str(x)[4:6], str(x)[6:8]])))
qy_data['frmWeek'] = qy_data['measDtStr'].map(lambda x : datetime.datetime(x[0], x[1], x[2]).isocalendar()[1])
qy_data

,ec,ph,cunt,otmsuplyqy,id,measDtStr,frmWeek
21,2.294236,5.768797,17.0,179.333333,9,"[2017, 9, 15]",37
24,2.693750,5.356250,5.0,210.576389,9,"[2017, 12, 20]",51
45,2.228261,5.850000,21.5,179.781746,9,"[2017, 9, 14]",37
85,2.228417,6.002417,20.0,163.439815,9,"[2017, 9, 13]",37
110,2.372917,5.931667,23.0,145.384470,9,"[2017, 9, 12]",37
...,...,...,...,...,...,...,...
6550,2.409259,5.609259,18.0,37.380952,349,"[2019, 6, 11]",24
6574,2.394444,5.642593,18.0,34.444444,349,"[2019, 6, 12]",24
6592,2.331481,5.664815,18.0,34.285714,349,"[2019, 6, 13]",24
6616,2.311111,5.683333,12.0,30.714286,349,"[2019, 6, 14]",24


In [6]:
qy_grw = pd.merge(grw_prd, qy_data, left_on=('id','frmWeek'), right_on=('id','frmWeek'), how='inner')
qy_grw = qy_grw[['id', 'frtstGrupp', 'flanGrupp', 'frtstCo', 'frmhsFclu', 'hvstGrupp', 'grwtLt', 'fcluHg', 'lefLt',
           'hvstCo', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'outtrn', 'ec', 'ph', 'cunt',
           'otmsuplyqy']]
qy_grw.to_csv("data/{}.csv".format('qy_grw'))
qy_grw

,id,frtstGrupp,flanGrupp,frtstCo,frmhsFclu,hvstGrupp,grwtLt,fcluHg,lefLt,hvstCo,frmWeek,lefCunt,lefBt,stemThck,outtrn,ec,ph,cunt,otmsuplyqy
0,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,43,0.00,0.000,0.0000,3140.0,2.104545,6.086364,8.5,185.000000
1,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,43,0.00,0.000,0.0000,3140.0,2.072378,6.017133,10.5,189.800000
2,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,43,0.00,0.000,0.0000,3140.0,2.064103,5.872051,11.0,181.166667
3,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,43,0.00,0.000,0.0000,3140.0,2.070833,5.843056,9.0,180.833333
4,9,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,0.0,43,0.00,0.000,0.0000,3140.0,2.067857,6.057143,11.5,166.527778
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8075,349,18.5,0.0,4.0,0.0,17.5,0.0,0.0,52.375,5.0,24,5.75,60.875,9.0125,10546.0,2.409259,5.609259,18.0,37.380952
8076,349,18.5,0.0,4.0,0.0,17.5,0.0,0.0,52.375,5.0,24,5.75,60.875,9.0125,10546.0,2.394444,5.642593,18.0,34.444444
8077,349,18.5,0.0,4.0,0.0,17.5,0.0,0.0,52.375,5.0,24,5.75,60.875,9.0125,10546.0,2.331481,5.664815,18.0,34.285714
8078,349,18.5,0.0,4.0,0.0,17.5,0.0,0.0,52.375,5.0,24,5.75,60.875,9.0125,10546.0,2.311111,5.683333,12.0,30.714286


In [15]:
'''
flanGrupp, frmhsFclu, fcluHg  #개화
frtstGrupp, frtstCo, fcluHg #착과
hvstGrupp, hvstCo fcluHg  #수확
grwtLt,lefLt, frmWeek lefCunt lefBt stemThck #기타
'''
#초반
qy_grw1 = qy_grw.loc[qy_grw['hvstGrupp'] == 0.0] 
qy_grw2 = qy_grw1.loc[qy_grw1['hvstCo'] == 0.0] 
qy_grw3 = qy_grw2.loc[qy_grw2['frtstGrupp'] == 0.0] 
qy_grw4 = qy_grw3.loc[qy_grw3['frtstGrupp'] == 0.0] 
qy_grw5 = qy_grw4.loc[qy_grw4['flanGrupp'] == 0.0] 
qy_grw6 = qy_grw5.loc[qy_grw5['frmhsFclu'] == 0.0] 
qy_grw7 = qy_grw6.loc[qy_grw6['fcluHg'] == 0.0] 
qy_st = qy_grw3[['grwtLt','lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'ec', 'ph', 'cunt', 'otmsuplyqy']]
qy_st.to_csv("data/{}.csv".format('qy_st'))
#개화
qy_grw1 = qy_grw.loc[qy_grw['flanGrupp'] != 0.0] 
qy_grw2 = qy_grw1.loc[qy_grw1['frmhsFclu'] != 0.0] 
qy_grw3 = qy_grw2.loc[qy_grw2['fcluHg'] != 0.0] 
qy_flw = qy_grw3[['flanGrupp', 'frmhsFclu', 'fcluHg', 'grwtLt','lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'ec', 'ph', 'cunt', 'otmsuplyqy']]
qy_flw.to_csv("data/{}.csv".format('qy_flw'))
#착과
qy_grw1 = qy_grw.loc[qy_grw['frtstGrupp'] != 0.0] 
qy_grw2 = qy_grw1.loc[qy_grw1['frtstCo'] != 0.0] 
qy_grw3 = qy_grw2.loc[qy_grw2['fcluHg'] != 0.0] 
qy_fru = qy_grw3[['frtstGrupp', 'frtstCo', 'fcluHg', 'grwtLt','lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'ec', 'ph', 'cunt', 'otmsuplyqy']]
qy_fru.to_csv("data/{}.csv".format('qy_fru'))
#수확
qy_grw1 = qy_grw.loc[qy_grw['hvstGrupp'] != 0.0] 
qy_grw2 = qy_grw1.loc[qy_grw1['hvstCo'] != 0.0] 
qy_grw3 = qy_grw2.loc[qy_grw2['fcluHg'] != 0.0] 
qy_hvst = qy_grw3[['hvstGrupp', 'hvstCo', 'fcluHg', 'grwtLt','lefLt', 'frmWeek', 'lefCunt', 'lefBt', 'stemThck', 'ec', 'ph', 'cunt', 'otmsuplyqy']]
qy_hvst.to_csv("data/{}.csv".format('qy_hvst'))

In [17]:
yap = []
for c in list(qy_st.columns):
    tmp = qy_st.loc[qy_st[c] != 0.0]
    if len(tmp) != 0:
        yap.append(c)
qy_st = qy_st[yap]
qy_st # 없는 데이터 지우기

,grwtLt,lefLt,frmWeek,lefCunt,lefBt,stemThck,ec,ph,cunt,otmsuplyqy
0,0.0,0.0,43,0.0,0.0,0.0,2.104545,6.086364,8.5,185.000000
1,0.0,0.0,43,0.0,0.0,0.0,2.072378,6.017133,10.5,189.800000
2,0.0,0.0,43,0.0,0.0,0.0,2.064103,5.872051,11.0,181.166667
3,0.0,0.0,43,0.0,0.0,0.0,2.070833,5.843056,9.0,180.833333
4,0.0,0.0,43,0.0,0.0,0.0,2.067857,6.057143,11.5,166.527778
...,...,...,...,...,...,...,...,...,...,...
7915,0.0,0.0,1,0.0,0.0,0.0,2.815385,5.666667,13.0,64.102564
7916,0.0,0.0,1,0.0,0.0,0.0,2.893333,5.576667,10.0,72.904762
7917,0.0,0.0,1,0.0,0.0,0.0,2.885185,5.677778,9.0,72.804233
7918,0.0,0.0,1,0.0,0.0,0.0,2.880000,5.673333,5.0,72.952381


## 1.2 환경정보 합친거 > 생산량